In [ ]:
import pandas as pd
import altair as alt
from datetime import datetime
import datetime

### Read CSV

In [ ]:
df_raw_dataset = pd.read_csv('C:\\Users\\JP\\Desktop\\Mohit\\Dataset\\Boonsong Dataset.csv')
sortdata_csv = pd.read_csv('C:\\Users\\JP\\Desktop\\Mohit\\Dataset\\sorted_data.csv')

### Sort Years

In [ ]:
sorted_years = []

for date in df_raw_dataset["sample date"].unique():
    date_obj = datetime.datetime.strptime(date, '%d-%b-%y')
    year = date_obj.year
    if year not in sorted_years:
        sorted_years.append(year)

# Task 1

## Finding 1 - Missing Data 

In [ ]:
 # Convert sample date to datetime format
df_raw_dataset['sample date'] = pd.to_datetime(df_raw_dataset['sample date'])

# Create a new column with the year of the sample date
df_raw_dataset['year'] = df_raw_dataset['sample date'].dt.year

# Aggregate by location, measure, and year to count the frequency of each combination
freq = df_raw_dataset.groupby(['location', 'measure', 'year']).size().reset_index(name='frequency')

list_location = freq['location'].unique().tolist()
list_measure = freq['measure'].unique().tolist()

for location in list_location:
    for measure in list_measure:
       result = freq.query('measure == "{measure}"')

freq_measures = df_raw_dataset.groupby(['measure', 'year']).size().reset_index(name='frequency')

for measure in list_measure:
    result = freq_measures.query(f'measure== "{measure}"')

In [ ]:
freq_measures = df_raw_dataset.groupby(['measure','location', 'year']).size().reset_index(name='frequency')

for measure in list_measure:
    result = freq_measures.query(f'measure == "{measure}"')
    year_diffs = result['year'].diff()
    gaps = result[(year_diffs != 1)]
    
    if not gaps.empty:
        result2 = freq_measures.query(f'measure== "{measure}"')

In [ ]:
freq_measures = df_raw_dataset.groupby(['measure','location', 'year']).size().reset_index(name='frequency')
measure_freq_inconsistent = []

for measure in list_measure:
    result = freq_measures.query(f'measure == "{measure}"')
    year_diffs = result['year'].diff()
    gaps = result[(year_diffs != 1) & (~year_diffs.isna())]

    if not gaps.empty:
        measure_freq_inconsistent.append(measure)

### Chart Finding 1

In [ ]:
measure_freq_inconsistent = []
plots = []

for location in list_location:
    for measure in list_measure:
        result = freq_measures.query(f'measure == "{measure}" and location=="{location}"')
        year_diffs = result['year'].diff()
        gaps = result[(year_diffs != 1) & (~year_diffs.isna())]

        if not gaps.empty:
            measure_freq_inconsistent.append(measure)
        
            # create a chart for the current measure
            chart = alt.Chart(result).mark_bar().encode(
                x='year:Q',
                y='frequency:Q'
            ).properties(
                title=f'{measure} at {location} has gaps in between years:',
                width=350,
                height=200
            )
        
            # add bars for gap years
            gap_years = list(gaps['year'].unique())
            gap_data = {'year': gap_years, 'frequency': [result[result['year'] == y]['frequency'].iloc[0] for y in gap_years]}
            gap_chart = alt.Chart(pd.DataFrame(gap_data)).mark_bar().encode(
                x='year:Q',
                y='frequency:Q'
            )
        
            chart += gap_chart
            plots.append(chart)
    print(len)
    
# concatenate all the plots in a grid
grid = alt.vconcat(*[alt.hconcat(*plots[i:i+7]) for i in range(0, len(plots), 7)])

# show the grid
grid.configure_axis(
    labelFontSize=8,
    titleFontSize=6,
).configure_title(
    fontSize=10
).configure_legend(
    titleFontSize=12,
    labelFontSize=8,
)

## Finding 2 - change in collection frequency

In [ ]:
 # Convert sample date to datetime format
df_raw_dataset['sample date'] = pd.to_datetime(df_raw_dataset['sample date'])
df_raw_dataset['year'] = df_raw_dataset['sample date'].dt.year
freq = df_raw_dataset.groupby(['location', 'measure', 'year']).size().reset_index(name='frequency')

list_location = freq['location'].unique().tolist()
list_measure = freq['measure'].unique().tolist()

for location in list_location:
    for measure in list_measure:
       result = freq.query('measure == "{measure}"')
    
freq_measures = df_raw_dataset.groupby(['location','measure', 'year']).size().reset_index(name='frequency')

# create a list of plots
plots = []
df_missing_year=pd.DataFrame()

for location in list_location:
    for measure in list_measure:
        result = freq_measures.query(f'location == "{location}" and measure == "{measure}"')
        year_diffs = result['year'].diff()
        gaps = result[(year_diffs != 1) & (~year_diffs.isna())]

        if not gaps.empty:
            new_row = {'location': location, 'measure': measure}
            new_df = pd.DataFrame(new_row, index=[0])
            df_missing_year=pd.concat([df_missing_year, new_df], ignore_index=True)


#create a new dataframe with locations and mesaure with missing gaps

merged_df = pd.merge(freq_measures, df_missing_year, on=['location', 'measure'], how='outer', indicator=True)

df_freq_distribution = merged_df[merged_df['_merge'] == 'left_only'].drop('_merge', axis=1)

df_freq_distribution = df_freq_distribution.groupby(['location', 'measure']).filter(lambda x: len(x) > 1)


### Chart Finding 2

In [ ]:
plots = []

list_location_freq = df_freq_distribution['location'].unique().tolist()
list_measure_freq = df_freq_distribution['measure'].unique().tolist()

for location in list_location_freq:
    for measure in list_measure_freq:
        result = df_freq_distribution.query(f'location == "{location}" and measure == "{measure}"')
        
        if not result.empty:
            chart = alt.Chart(result).mark_bar().encode(
                x='year:O',
                y='frequency:Q'
            ).properties(
                title=f'{measure} in {location}:'
                
            )
            plots.append(chart)
    
grid = alt.vconcat(*[alt.hconcat(*plots[i:i+10]) for i in range(0, len(plots), 10)])

# show the grid
grid.configure_axis(
    labelFontSize=10,
    titleFontSize=9,
).configure_title(
    fontSize=5
).configure_legend(
    titleFontSize=9,
    labelFontSize=5,
)

## Finding 3 - Unrealistic Values

In [ ]:
chart_all=[]
for measure in ['Copper','Fecal coliforms','Iron','Manganese','Total coliforms']:
    # Filter the data for the current measure
    filtered_data = df_raw_dataset[df_raw_dataset["measure"] == measure]
    # Create a bar chart for the current measure using Altair
    chart = alt.Chart(filtered_data).mark_point().encode(
        x=alt.X("year(sample date):O", title="Years", scale=alt.Scale(domain=[1998,1999,2000,2001,2002,2003,2004,
                                                                              2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016])),
        y=alt.Y("value:Q", title="Value"),
        color='location',
        tooltip=["measure", "value", "id","location","sample date:T"]
    ).properties(
        title=f"{measure} Measurement",
        width=600,
        height=400
    )
    chart
    chart_all.append((chart))

    # Combine charts horizontally
combined_chart = alt.hconcat(*chart_all)
    # Display chart
combined_chart.display()

# Task 2 - Describe trends and anomalies with respect to chemical contamination

### 1. Trends: changes over time and/or sensor site

In [ ]:
alt.data_transformers.disable_max_rows()

measurements = ['Potassium', 'Sulphates', 'Chemical Oxygen Demand (Cr)']
charts_list = []
    # Loop through measurements
for measure in measurements:
        # Filter data by measurement and location
        df_filtered = df_raw_dataset[(df_raw_dataset['measure'] == measure) & (df_raw_dataset['location'].isin(['Kohsoom', 'Boonsri']))]
        # Create base chart
        base = alt.Chart(df_filtered, title=measure).encode(
            x=alt.X('sample date:T', title='Year'),
            y=alt.Y('value:Q', title=measure),
            tooltip=['measure','location']
        )
        # Create selection 
        selection = alt.selection_single(
            fields=['location'],
            clear=False,
            on='mouseover'
        )

        # Create chart with points colored by location
        points = base.mark_point(filled=True, size=50).encode(
            color=alt.condition(selection, 'location:N', alt.value('lightgray'), legend=None)
        ).add_selection(selection)

        # Create chart with lines colored by location
        lines = base.mark_line().encode(
            color=alt.Color('location:N', legend=None)
        ).transform_filter(
            selection
        )
        legend = alt.Chart(df_filtered).mark_point().encode(
        alt.Y('location:N').axis(orient='right'),
        color='location').add_params(
        selection
        )
        # Combine points and lines charts
        chart = alt.layer(points, lines).properties(width=300, height=200) 
        # Add chart to list
        charts_list.append((chart|legend))
    # Combine charts horizontally
combined_chart = alt.hconcat(*charts_list)
    # Display chart
combined_chart.display()

### 2. Anomalies: sudden change over time or one site significantly different from others.

In [ ]:
brush = alt.selection_interval()

base_chart = alt.Chart(sortdata_csv).mark_line().encode(
    x='location:N',
    y=alt.Y('sum(measure):Q', axis=alt.Axis(title='Sum of Value')),
    
)

missing_year = []
for measure in sortdata_csv["measure"].unique():
    selected_measure = sortdata_csv[sortdata_csv["measure"] == measure]
    
    #Convert to year
    for date in selected_measure["sample date"].unique():
        date_obj = datetime.datetime.strptime(date, '%d-%b-%y')
        year = date_obj.year
        
        if year not in missing_year:
         missing_year.append(year)

    if len(missing_year) == len(sorted_years):
        # Create chart
        chart1 = alt.Chart(selected_measure).mark_circle().encode(
        x=alt.X("location"),
        y=alt.Y("measure"),
        tooltip=["measure", "value", "id","location"],
        size='average(value)', 
        color=alt.condition(brush, 'average(value):Q', alt.value('lightgray'))
        ).properties(
            title=f"{measure} Measurement",
            width=400,
            height=200
        ).add_selection(
        brush )
        
        chart2 = base_chart.encode(
        y=alt.Y('average(value):Q', axis=alt.Axis(title='Sum of {}'.format(measure))),
        color=alt.condition(brush, alt.value('lightgreen'), alt.value('lightgray'))
        ).transform_filter(
        alt.FieldOneOfPredicate(field='measure', oneOf=[measure])
        ).properties(
            title=f"{measure} Measurement",
            width=400,
            height=200
        ).add_selection(
        brush )
        display(chart1 | chart2)
